In [233]:
!pip install selenium
!pip install bs4

In [234]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.common.by import By

In [235]:
main_link = 'https://www.tokopedia.com/search?st=&q=handphone&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource='

path = 'data\chrome.exe'

chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('disable-notifications')
chrome_options.add_argument('--disable-infobars')
chrome_options.binary_location = path

driver = webdriver.Chrome(options=chrome_options)
time.sleep(10)
driver.get(main_link)

<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11448\963250374.py:3: SyntaxWarning: invalid escape sequence '\c'
  path = 'data\chrome.exe'


In [236]:
html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
soup = BeautifulSoup(html, "html.parser")

In [237]:
shop_name, product_name, product_price, sold, brand, ram = [], [], [], [], [], []

def scroll_to_bottom(driver, scroll_pause_time=1, scroll_increment=1000):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script(f"window.scrollBy(0, {scroll_increment});")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def wait_for_elements(class_name, timeout=10):
    try:
        elements = WebDriverWait(driver, timeout).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, class_name))
        )
        return elements
    except:
        return []

def scrape_page():
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    product_elements = soup.find_all('div', class_='css-1nylpq2')
    if not product_elements:
        print("Product name not found.")
    else:
        found_product = False
        for i in product_elements:
            product_name.append(i.text.strip())
            print(f"Product name: {i.text.strip()}")
            found_product = True
        if not found_product:
            product_name.append("Null")
            print("No Product data found, setting Product to Null.")

    price_elements = soup.find_all('div', class_='price')
    if not price_elements:
        print("Product price not found.")
    else:
        found_price = False
        for i in price_elements:
            product_price.append(i.text.strip())
            print(f"Product Price: {i.text.strip()}")
            found_price = True
        if not found_price:
            product_price.append("Null")
            print("No Price data found, setting Price to Null.")

    shop_elements = soup.find_all('h2', class_='css-1wdzqxj-unf-heading e1qvo2ff2')
    if not shop_elements:
        print("Store name not found.")
    else:
        found_shop = False
        for i in shop_elements:
            shop_name.append(i.text.strip())
            print(f"Store name: {i.text.strip()}")
            found_shop = True
        if not found_shop:
            shop_name.append("Null")
            print("No shop data found, setting shop to Null.")

    sold_elements = soup.find_all('div', class_='items')
    if not sold_elements:
        print("The number of items sold was not found.")
    else:
        found_sold = False
        for i in sold_elements:
            sold.append(i.text.strip())
            print(f"Sold: {i.text.strip()}")
            found_sold = True
        if not found_sold:
            sold.append("Null")
            print("No Sold data found, setting Sold to Null.")
    
    brand_elements = soup.find_all('td', class_='css-kmh5l0')[2]
    if not brand_elements:
        print("The number of items brand was not found.")
    else:
        found_brand = False
        for i in brand_elements:
            brand.append(i.text.strip())
            print(f"Brand: {i.text.strip()}")
            found_brand = True
        if not found_brand:
            brand.append("Null")
            print("No Brand data found, setting Brand to Null.")
    
    ram_elements = soup.find_all('td', class_='css-kmh5l0')[1]
    if not ram_elements:
        print("The number of items RAM was not found.")
    else:
        found_gb = False
        for i in ram_elements:
            text = i.text.strip()
            if "GB" in text:
                ram.append(text)
                print(f"Ram: {i.text.strip()}")
                found_gb = True
        if not found_gb:
            ram.append("Null")
            print("No RAM data with 'GB' found, setting RAM to Null.")

def click_next_page():
    try:
        next_button = driver.find_element(By.CLASS_NAME, 'css-bugrro-unf-pagination-item')
        if next_button:
            next_button.click()
            time.sleep(10)
            return True
    except:
        print("The next page button is not found or cannot be clicked.")
        return False

def click_specification_tab():
    try:
        spec_button = driver.execute_script("return document.querySelector('#pdp_comp-product_detail > div:nth-child(2) > div.css-1bl48du-unf-tab-wrapper.ehv0kkf0 > div > button:nth-child(2)')")
        if spec_button:
            driver.execute_script("arguments[0].click();", spec_button)
            print("Clicked on 'Spesifikasi' tab.")
        else:
            print("Specification button not found via JavaScript.")
    except Exception as e:
        print(f"Error clicking on 'Spesifikasi' tab: {e}")

time.sleep(10)

menu_elements = wait_for_elements('css-5wh65g')

max_pages = 2  # Change this according to the desired number of pages
current_page = 1  # First page

while current_page <= max_pages:
    if not menu_elements:
        print("Menu not found, exit.")
        break
    else:
        total_elements = len(menu_elements)
        max_menu_elements = 2  # The maximum amount of data you want to process on each page
        print(f"Total menus found: {total_elements}, will process maximum {max_menu_elements} menu on the page {current_page}")

        for index in range(min(total_elements, max_menu_elements)):
            try:
                menu_elements = wait_for_elements('css-5wh65g')
                print(f"Clicking on the menu {index + 1} from maximum {max_menu_elements} menu")
                menu_elements[index].click()
                
                time.sleep(10)
                
                click_specification_tab()
                time.sleep(10)
                scrape_page()
 
                time.sleep(10)

                driver.back()
                time.sleep(10)

                menu_elements = wait_for_elements('css-5wh65g')
                
                scroll_to_bottom(driver, scroll_pause_time=3, scroll_increment=1000)

            except Exception as e:
                print(f"Error when clicking on element {index + 1}: {e}")
                break  

    if current_page >= max_pages or not click_next_page():
        print(f"There are no next pages or page limits ({max_pages}) has been achieved. Finished.")
        break

    current_page += 1

driver.quit()

Total menus found: 85, will process maximum 2 menu on the page 1
Clicking on the menu 1 from maximum 2 menu
Clicked on 'Spesifikasi' tab.
Product name: Samsung Galaxy S24 FE [8/256] - [Blue, Graphite, Gray] Smartphone AI, AI Phone, Handphone AI, Kamera 50MP, Layar besar, RAM besar, Baterai tahan lama - Blue
Product Price: Rp9.999.000
Store name: Samsung Official Store
Sold: Terjual 90+•Diskusi (7)
Brand: Samsung
Ram: 8 GB
Clicking on the menu 2 from maximum 2 menu
Clicked on 'Spesifikasi' tab.
Product name: OPPO A3x 4GB(+4GB)/64GB (Garansi Resmi) - Biru
Product Price: Rp1.599.000
Store name: OPPO OFFICIAL STORE
Sold: Terjual 100+•bintang 4.9 (72 rating)•Diskusi (5)
Brand: Garansi Merek (Resmi)
No RAM data with 'GB' found, setting RAM to Null.
Total menus found: 85, will process maximum 2 menu on the page 2
Clicking on the menu 1 from maximum 2 menu
Clicked on 'Spesifikasi' tab.
Product name: Samsung Galaxy S24 FE [8/256] - [Blue, Graphite, Gray] Smartphone AI, AI Phone, Handphone AI, K

In [238]:
min_length = min(len(product_name), len(product_price), len(shop_name), len(sold), len(brand), len(ram))

shop_name = shop_name[:min_length]
product_name = product_name[:min_length]
product_price = product_price[:min_length]
sold = sold[:min_length]
brand = brand[:min_length]
ram = ram[:min_length]

data = {
    'shop_name': shop_name,
    'product_name': product_name,
    'product_price': product_price,
    'sold': sold,
    'brand': brand,
    'ram': ram
}
df = pd.DataFrame(data)
df.to_csv('data/scraping_results.csv', index=False)

In [239]:
import pandas as pd
data = pd.read_csv("data/scraping_results.csv")
data

,shop_name,product_name,product_price,sold,brand,ram
0,Samsung Official Store,"Samsung Galaxy S24 FE [8/256] - [Blue, Graphit...",Rp9.999.000,Terjual 90+•Diskusi (7),Samsung,8 GB
1,OPPO OFFICIAL STORE,OPPO A3x 4GB(+4GB)/64GB (Garansi Resmi) - Biru,Rp1.599.000,Terjual 100+•bintang 4.9 (72 rating)•Diskusi (5),Garansi Merek (Resmi),Null
2,Samsung Official Store,"Samsung Galaxy S24 FE [8/256] - [Blue, Graphit...",Rp9.999.000,Terjual 90+•Diskusi (7),Samsung,8 GB
3,OPPO OFFICIAL STORE,OPPO A3x 4GB(+4GB)/64GB (Garansi Resmi) - Biru,Rp1.599.000,Terjual 100+•bintang 4.9 (72 rating)•Diskusi (5),Garansi Merek (Resmi),Null
